# Neteja i preprocessament dels conjunts de dades d'ADS-B i ICA

Aquest notebook descriu el procés de **neteja, transformació i preprocessament** dels conjunts de dades utilitzats en el treball, basant-se en les conclusions obtingudes durant l'anàlisi exploratòria prèvia.

### Objectiu
L'objectiu d'aquest notebook és:
- Corregir problemes de qualitat detectats en l'anàlisi exploratòria.
- Unificar formats temporals i espacials.
- Afegir variables derivades necessàries per a l'anàlisi (colors ICA, grid geogràfic, etc.).
- Preparar les dades per a la visualització (mapa) i l'anàlisi de correlacions.
- Garantir la coherència i la traçabilitat dels registres.

### Abast
En aquest notebook es realitzen tasques de:
- **Neteja**: gestió de valors nuls, conversió de tipus, filtratge de registres.
- **Preprocessament**: creació de variables derivades, filtratge geogràfic, creació de grid espacial.

No es realitza encara cap anàlisi estadística avançada ni modelització.

## 1. Carregar dades brutes

In [1]:
# Importar les biblioteques necessàries
from pathlib import Path
import pandas as pd
import numpy as np

/tmp/ipykernel_40/1028428549.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Llegir els fitxers CSV
adsb_path = Path("../data/raw/adsb/adsb_raw.csv")
df_adsb = pd.read_csv(adsb_path)
 
ica_path = Path("../data/merged/ica_merged_raw.csv")
df_ica = pd.read_csv(ica_path)

print(f"Fitxer ADS-B: {len(df_adsb)} registres, {len(df_adsb.columns)} columnes")
print(f"Fitxer ICA: {len(df_ica)} registres, {len(df_ica.columns)} columnes")

Fitxer ADS-B: 137574 registres, 13 columnes
Fitxer ICA: 16195 registres, 9 columnes


## 2. Neteja del conjunt de dades ADS-B

A partir dels resultats de l'anàlisi exploratòria, el conjunt de dades ADS-B presenta una estructura coherent i una qualitat general elevada. Tot i això, s'han identificat diversos aspectes a millorar abans de procedir a la seva utilització analítica:

- Existència de valors nuls en variables categòriques com `category`.
- Necessitat d'unificar el format temporal per facilitar agregacions.
- Presència de valors textuals amb possibles espais en blanc.
- Preparació d'una variable temporal agregada per a l'anàlisi horària.

Les operacions següents s'han dissenyat per preservar la quantitat d'informació més gran possible, evitant l'eliminació innecessària de registres.


In [3]:
# Netejar columna 'flight' (blancs → NaN)
if 'flight' in df_adsb.columns:
    df_adsb['flight'] = df_adsb['flight'].astype(str).str.strip().replace('', np.nan)

# Omplir nuls de 'category' amb 'Unknown'
if 'category' in df_adsb.columns:
    null_cat = df_adsb['category'].isna().sum()
    df_adsb['category'] = df_adsb['category'].fillna('Unknown')

# Convertir 'timestamp' a datetime
if 'timestamp' in df_adsb.columns:
    df_adsb['timestamp'] = pd.to_datetime(df_adsb['timestamp'])
    df_adsb['hora'] = df_adsb['timestamp'].dt.floor('h')
elif 'hora' in df_adsb.columns:
    df_adsb['hora'] = pd.to_datetime(df_adsb['hora'])

print("Resum ADS-B:")
print(f"- Registres: {len(df_adsb):,}")
print(f"- Nuls 'category' convertits a 'Unknown': {null_cat}")
print(f"- Rang temporal: {df_adsb['timestamp'].min()} a {df_adsb['timestamp'].max()}")
print(f"- Rang altitud: {df_adsb['alt_baro'].min()} - {df_adsb['alt_baro'].max()} ft")
print(f"- Latituds: {df_adsb['lat'].min():.2f} a {df_adsb['lat'].max():.2f}")
print(f"- Longituds: {df_adsb['lon'].min():.2f} a {df_adsb['lon'].max():.2f}")

Resum ADS-B:
- Registres: 137,574
- Nuls 'category' convertits a 'Unknown': 1085
- Rang temporal: 2025-10-11 06:00:12.619226 a 2025-10-12 06:03:22.696339
- Rang altitud: 10000 - 49000 ft
- Latituds: 31.54 a 49.55
- Longituds: -15.49 a 8.32


### Resultats de la neteja del conjunt ADS-B

Després del procés de neteja i transformació, el conjunt de dades ADS-B presenta les següents característiques:

- No s'ha eliminat cap registre, mantenint la totalitat de l'activitat aèria observada.
- Els valors nuls de la variable `category` s'han reassignat a la categoria `Unknown`, preservant la informació i evitant la pèrdua de registres.
- La variable temporal `timestamp` s'ha convertit a format datetime, permetent una manipulació temporal correcta.
- S'ha creat una nova variable `hora`, arrodonida a l'hora, amb l'objectiu de facilitar futures agregacions temporals.
- El rang espacial i d'altitud dels vols es manté coherent amb l'activitat aèria esperada sobre el territori d'estudi.

## 3. Neteja del conjunt de dades ICA (qualitat de l'aire)

L'anàlisi exploratòria del conjunt ICA ha posat de manifest diversos aspectes crítics que requereixen una neteja específica abans de la seva utilització:

- Existència de valors nuls en la variable `indice`, considerada la variable principal del dataset.
- Presència de valors nuls en la variable explicativa `debido_a`.
- Necessitat d'homogeneïtzar els noms de les coordenades geogràfiques.
- Existència de registres corresponents a estacions inactives.

Les decisions preses en aquest procés busquen garantir la fiabilitat de les mesures i la coherència temporal i espacial del conjunt de dades.

In [4]:
# Eliminar registres amb 'indice' null
n_before = len(df_ica)
null_index = df_ica['indice'].isna().sum()
df_ica = df_ica.dropna(subset=['indice'])

# Omplir nulls de 'debido_a'
null_debido = df_ica['debido_a'].isna().sum()
df_ica['debido_a'] = df_ica['debido_a'].fillna('No especificat')

# Convertir 'fecha' a datetime i renombrar a 'hora'
df_ica['hora'] = pd.to_datetime(df_ica['fecha'])
df_ica = df_ica.drop(columns=['fecha'])

# Renombrar coordenades per unificar amb ADS-B
df_ica = df_ica.rename(columns={'latitud': 'lat', 'longitud': 'lon'})

# Filtrar estacions actives
n_inactive = len(df_ica[df_ica['activa'] == False])
df_ica = df_ica[df_ica['activa'] == True]

print("Resum ICA:")
print(f"- Registres finals: {len(df_ica):,}")
print(f"- S'han eliminat {null_index} registres amb 'indice' null ({null_index/n_before*100:.2f}%)")
print(f"- S'han convertit {null_debido} valors nuls de 'debido_a' a 'No especificado'")
print(f"- S'han filtrat un total de {n_before - len(df_ica)} estacions inactives")
print(f"- Estacions úniques: {df_ica['cod_estacion'].nunique()}")
print(f"- Rang temporal: {df_ica['hora'].min()} a {df_ica['hora'].max()}")
print(f"- Índex: {df_ica['indice'].min()} - {df_ica['indice'].max()}")

Resum ICA:
- Registres finals: 13,850
- S'han eliminat 1670 registres amb 'indice' null (10.31%)
- S'han convertit 646 valors nuls de 'debido_a' a 'No especificado'
- S'han filtrat un total de 2345 estacions inactives
- Estacions úniques: 554
- Rang temporal: 2025-10-11 04:00:00 a 2025-10-12 06:00:00
- Índex: 0.0 - 30.0


### Resultats de la neteja del conjunt ICA

Un cop aplicades les operacions de neteja, el conjunt de dades ICA presenta:

- Eliminació dels registres amb valor nul a la variable `indice`, ja que no aporten informació útil per a l'anàlisi de la qualitat de l'aire.
- Reassignació dels valors nuls de `debido_a` a la categoria `No especificat`, evitant la pèrdua de registres.
- Conversió de la variable temporal a format datetime i creació de la variable `hora`.
- Renombrat de les coordenades geogràfiques per unificar-les amb el conjunt ADS-B.
- Filtratge de les estacions inactives per garantir la validesa de les mesures.

## 4. Preprocessament: Creació de variables derivades

Un cop netejades les dades, es procedeix a la creació de variables derivades necessàries per a la visualització i l'anàlisi posterior:

### 3.1. Categories i colors oficials de l'Índex Nacional de Qualitat de l'Aire (ICA)

Segons la normativa vigent del Ministerio para la Transición Ecológica y el Reto Demográfico (MITECO), l'**Índex Nacional de Qualitat de l'Aire (ICA)** és un índex **categòric** (no numèric escalat) que classifica la qualitat de l'aire en 6 nivells més la categoria "Sense dades". La categoria final d'una estació es determina per **la pitjor categoria** entre els contaminants disponibles (el que presenta el major risc sanitari en aquell moment), segons les bandes de concentració definides oficialment.

Les categories i els colors RGB oficials són els següents:

* **Bona**: rgb(56, 162, 206) – blau cel
* **Raonablement bona**: rgb(50, 161, 94) – verd
* **Regular**: rgb(241, 229, 73) – groc
* **Desfavorable**: rgb(200, 52, 65) – vermell
* **Molt desfavorable**: rgb(110, 22, 29) – granat
* **Extremadament desfavorable**: rgb(162, 91, 164) – morat
* **Sense dades**: rgb(85, 89, 93) – gris

Aquestes categories i colors són els utilitzats actualment al visor oficial del MITECO i provenen directament de la metodologia aprovada per la **Resolución de 2 de septiembre de 2020**, de la Dirección General de Calidad y Evaluación Ambiental (BOE-A-2020-10426), que modifica l'annex de l'Ordre TEC/351/2019, de 18 de març. Aquesta resolució segueix vigent el 2026 sense modificacions documentades en les bandes ni en la metodologia de càlcul.

**Com es determina la categoria** 

L'ICA es calcula aplicant les bandes de concentració (en µg/m³) definides per cada contaminant clau: SO₂, PM₂.₅, PM₁₀, O₃ i NO₂. Segons la normativa, les franges són les següents:

| SO₂ (µg/m³) | PM₂,₅ (µg/m³) | PM₁₀ (µg/m³) | O₃ (µg/m³) | NO₂ (µg/m³) | CATEGORIA DE L'ÍNDEX |
| --- | --- | --- | --- | --- | --- |
| 0 - 100 | 0 - 10 | 0 - 20 | 0 - 50 | 0 - 40 | **Bona** |
| 101 - 200 | 11 - 20 | 21 - 40 | 51 - 100 | 41 - 90 | **Raonablement bona** |
| 201 - 350 | 21 - 25 | 41 - 50 | 101 - 130 | 91 - 120 | **Regular** |
| 351 - 500 | 26 - 50 | 51 - 100 | 131 - 240 | 121 - 230 | **Desfavorable** |
| 501 - 750 | 51 - 75 | 101 - 150 | 241 - 380 | 231 - 340 | **Molt desfavorable** |
| 751 - 1250 | 76 - 800 | 151 - 1200 | 381 - 800 | 341 - 1000 | **Extremadament desfavorable** |

La categoria final és la **pitjor** (la més avançada en gravetat) entre tots els contaminants disponibles a l'estació en aquell moment. En les dades horàries descarregades, només es proporciona la concentració (`indice`) del contaminant responsable (`debido_a`), que és el que ha determinat la categoria en aquella hora.

**Referències oficials**  
- Resolució BOE-A-2020-10426 (2 de setembre 2020): [BOE](https://www.boe.es/diario_boe/txt.php?id=BOE-A-2020-10426)

Aquestes categories s'han assignat al dataframe utilitzant el valor de concentració (`indice`) del contaminant dominant (`debido_a`), aplicant les bandes oficials per obtenir la categoria i el color RGB corresponent. Això garanteix que la visualització sigui fidel a la normativa vigent.

In [5]:
# Ordre oficial de categories (millor a pitjor)
categories_order = [
    'Bona', 'Raonablement bona', 'Regular', 
    'Desfavorable', 'Molt desfavorable', 'Extremadament desfavorable',
    'Sin datos'  # al final perquè és "desconegut"
]

# Colors oficials MITECO (RGB exactes del visor)
color_map = {
    'Bona': 'rgb(56, 162, 206)',
    'Raonablement bona': 'rgb(50, 161, 94)',
    'Regular': 'rgb(241, 229, 73)',
    'Desfavorable': 'rgb(200, 52, 65)',
    'Molt desfavorable': 'rgb(110, 22, 29)',
    'Extremadament desfavorable': 'rgb(162, 91, 164)',
    'Sin datos': 'rgb(85, 89, 93)'
}

# Funció per obtenir categoria segons contaminant i valor (bandes del BOE 2020)
def get_category_pollutant(value, pollutant):
    if pd.isna(value):
        return None
    # Ajusta segons contaminant
    if pollutant == 'SO2':
        if value <= 100: return 'Bona'
        elif value <= 200: return 'Raonablement bona'
        elif value <= 350: return 'Regular'
        elif value <= 500: return 'Desfavorable'
        elif value <= 750: return 'Molt desfavorable'
        else: return 'Extremadament desfavorable'
    elif pollutant in ['PM25', 'PM2.5']:  # accepta 'PM2.5' del CSV
        if value <= 10: return 'Bona'
        elif value <= 20: return 'Raonablement bona'
        elif value <= 25: return 'Regular'
        elif value <= 50: return 'Desfavorable'
        elif value <= 75: return 'Molt desfavorable'
        else: return 'Extremadament desfavorable'
    elif pollutant == 'PM10':
        if value <= 20: return 'Bona'
        elif value <= 40: return 'Raonablement bona'
        elif value <= 50: return 'Regular'
        elif value <= 100: return 'Desfavorable'
        elif value <= 150: return 'Molt desfavorable'
        else: return 'Extremadament desfavorable'
    elif pollutant in ['O3', 'O₃']:
        if value <= 50: return 'Bona'
        elif value <= 100: return 'Raonablement bona'
        elif value <= 130: return 'Regular'
        elif value <= 240: return 'Desfavorable'
        elif value <= 380: return 'Molt desfavorable'
        else: return 'Extremadament desfavorable'
    elif pollutant == 'NO2':
        if value <= 40: return 'Bona'
        elif value <= 90: return 'Raonablement bona'
        elif value <= 120: return 'Regular'
        elif value <= 230: return 'Desfavorable'
        elif value <= 340: return 'Molt desfavorable'
        else: return 'Extremadament desfavorable'
    return None

# Funció principal per al teu dataframe
def get_ica_from_indice_debido(row):
    pol = row['debido_a']
    val = row['indice']
    
    if pol == 'No especificat' or pd.isna(val) or pd.isna(pol):
        cat = 'Sin datos'
    else:
        # Mapeja noms del CSV als de la funció
        pol_key = pol.replace('2.5', '25')  # PM2.5 → PM25
        cat = get_category_pollutant(val, pol_key)
        if cat is None:
            cat = 'Sin datos'  # contaminant desconegut
    
    color = color_map.get(cat, 'rgb(85, 89, 93)')  # default gris
    return color, cat

# Aplicar (després de la teva neteja)
df_ica[['ColorICA', 'QualitatText']] = df_ica.apply(get_ica_from_indice_debido, axis=1, result_type='expand')

# Convertir a Categorical per ordenar automàticament (evita l'error!)
df_ica['QualitatText'] = pd.Categorical(
    df_ica['QualitatText'],
    categories=categories_order,
    ordered=True
)

print("Distribució de qualitat de l'aire:\n")
print(df_ica['QualitatText'].value_counts().sort_index())

Distribució de qualitat de l'aire:

QualitatText
Bona                          13647
Raonablement bona                82
Regular                           0
Desfavorable                      0
Molt desfavorable                 0
Extremadament desfavorable        0
Sin datos                       121
Name: count, dtype: int64


### 3.2. Filtratge geogràfic: Àmbit peninsular i Balears
Per tal d'optimitzar el processament de les dades i simplificar la representació cartogràfica, s'ha definit un marc geogràfic que comprèn exclusivament el territori de l'Espanya peninsular i les Illes Balears.

S'ha decidit excloure les Illes Canàries d'aquest estudi a causa de la complexitat tècnica que implica la seva representació en un mateix visor (degut a la gran distància geogràfica i la diferència de fus horari). Atès el plantejament actual del codi i els objectius del treball, s'ha prioritzat la continuïtat territorial per mantenir la claredat en l'anàlisi de dades.

**Coordenades de filtratge:**

- Latitud: 35.0° N - 44.5° N

- Longitud: -10.0° E - 5.0° E

Aquest rang assegura que qualsevol registre amb coordenades fora d'aquests límits (errors de geolocalització o estacions en territoris no analitzats) sigui descartat automàticament durant el preprocessament.

In [6]:
# Definir límits geogràfics
spain_bounds = {
    'lat_min': 35.0,
    'lat_max': 44.5,
    'lon_min': -10.0,
    'lon_max': 5.0
}

# Eliminar registres amb coordenades nul·les
n_adsb_before = len(df_adsb)
n_ica_before = len(df_ica)

df_adsb = df_adsb.dropna(subset=['lat', 'lon'])
df_ica = df_ica.dropna(subset=['lat', 'lon'])

# Aplicar filtratge geogràfic
df_adsb = df_adsb[
    (df_adsb['lat'].between(spain_bounds['lat_min'], spain_bounds['lat_max'])) &
    (df_adsb['lon'].between(spain_bounds['lon_min'], spain_bounds['lon_max']))
].copy()

df_ica = df_ica[
    (df_ica['lat'].between(spain_bounds['lat_min'], spain_bounds['lat_max'])) &
    (df_ica['lon'].between(spain_bounds['lon_min'], spain_bounds['lon_max']))
].copy()

print(f"ADS-B: {n_adsb_before:,} → {len(df_adsb):,} ({n_adsb_before - len(df_adsb)} eliminats)")
print(f"ICA: {n_ica_before:,} → {len(df_ica):,} ({n_ica_before - len(df_ica)} eliminats)")

ADS-B: 137,574 → 75,989 (61585 eliminats)
ICA: 13,850 → 12,625 (1225 eliminats)


### 3.3. Creació de grid espacial per a l'anàlisi ADS-B
Per facilitar l'agregació espacial dels vols i la seva posterior correlació amb les estacions ICA, es crea una quadrícula (grid) que divideix el territori en cel·les de 0.5° x 0.5°.

Cada registre ADS-B s'assigna a una cel·la identificada per:
- cell_id: identificador únic de la cel·la.
- lat_centre, lon_centre: coordenades del centre de la cel·la.

Aquesta estratègia permet:
- Agrupar vols en zones geogràfiques.
- Comparar activitat aèria amb estacions ICA properes.
- Visualitzar densitat de trànsit aeri.

In [7]:
# Definir mida del grid (0.5 graus)
GRID_SIZE = 0.5

# Crear bins per latitud i longitud
lat_bins = np.arange(spain_bounds['lat_min'], spain_bounds['lat_max'] + GRID_SIZE, GRID_SIZE)
lon_bins = np.arange(spain_bounds['lon_min'], spain_bounds['lon_max'] + GRID_SIZE, GRID_SIZE)

# Assignar cada registre a una cel·la del grid
df_adsb['lat_cell'] = pd.cut(df_adsb['lat'], bins=lat_bins, labels=False, include_lowest=True)
df_adsb['lon_cell'] = pd.cut(df_adsb['lon'], bins=lon_bins, labels=False, include_lowest=True)

# Crear identificador únic per a cada cel·la
df_adsb['cell_id'] = df_adsb['lat_cell'].astype(str) + '_' + df_adsb['lon_cell'].astype(str)

# Calcular el centre de cada cel·la
df_adsb['lat_centre'] = lat_bins[df_adsb['lat_cell'].astype(int)] + GRID_SIZE/2
df_adsb['lon_centre'] = lon_bins[df_adsb['lon_cell'].astype(int)] + GRID_SIZE/2

print(f"Mida del grid: {GRID_SIZE}° x {GRID_SIZE}°")
print(f"Total de cel·les úniques: {df_adsb['cell_id'].nunique():,}")
print(f"Registres amb cell_id assignat: {df_adsb['cell_id'].notna().sum():,}")

Mida del grid: 0.5° x 0.5°
Total de cel·les úniques: 563
Registres amb cell_id assignat: 75,989


## 5. Resum final del preprocessament

Després de la neteja i el preprocessament, els conjunts de dades presenten les següents característiques.

In [8]:
print("ADS-B (vols):")
print(f"- Registres finals: {len(df_adsb):,}")
print(f"- Columnes: {len(df_adsb.columns)}")
print(f"- Vols únics: {df_adsb['hex'].nunique():,}")
print(f"- Rang temporal: {df_adsb['hora'].min()} a {df_adsb['hora'].max()}")
print(f"- Cel·les grid úniques: {df_adsb['cell_id'].nunique():,}")

print("\nICA (qualitat aire):")
print(f"- Registres finals: {len(df_ica):,}")
print(f"- Columnes: {len(df_ica.columns)}")
print(f"- Estacions úniques: {df_ica['cod_estacion'].nunique()}")
print(f"- Rang temporal: {df_ica['hora'].min()} a {df_ica['hora'].max()}")
print(f"- Rang ICA: {df_ica['indice'].min():.1f} - {df_ica['indice'].max():.1f}")

ADS-B (vols):
- Registres finals: 75,989
- Columnes: 19
- Vols únics: 2,943
- Rang temporal: 2025-10-11 06:00:00 a 2025-10-12 06:00:00
- Cel·les grid úniques: 563

ICA (qualitat aire):
- Registres finals: 12,625
- Columnes: 11
- Estacions úniques: 505
- Rang temporal: 2025-10-11 04:00:00 a 2025-10-12 06:00:00
- Rang ICA: 0.0 - 30.0


## 6. Guardar fitxers processats

Els fitxers nets i preprocessats es guarden per a la seva utilització posterior en:
- Creació del mapa interactiu.
- Anàlisi de correlacions.
- Dashboard de visualització amb Tableau.

In [9]:
# Guardar CSV nets
df_adsb.to_csv("../data/processed/adsb/adsb_clean.csv", index=False)
df_ica.to_csv("../data/processed/ica/ica_clean.csv", index=False)

### Conclusió

El procés de neteja i preparació de dades ha permès obtenir dos conjunts de dades coherents, homogenis i alineats temporalment i espacialment.

Aquest pas ha estat fonamental per garantir que les anàlisis posteriors i els possibles encreuaments entre activitat aèria i qualitat de l'aire es realitzin sobre dades fiables i comparables.